# UM Data Exploration

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
DATASET_FILE = '../data/processed/um.csv'
subscore_cols = ['evidence','suggestion','connection']

In [ ]:
# Load the file
df = (pd.read_csv(DATASET_FILE)
        .rename({'Unnamed: 0': 'file_index'}, axis=1))

# QuAL

## Exploration

The number of items for each QuAL level, including NaN QuALs

In [ ]:
df['qual'].value_counts(dropna=False).sort_index()

Descriptive statistics

In [ ]:
df['qual'].describe().reset_index().T

Histogram

In [ ]:
plt.figure(figsize=(5,3))
df['qual'].value_counts().sort_index().plot.bar()

Descriptives by clerkship

In [ ]:
df.groupby('clerkship')['qual'].describe()

Average score by clerkship

In [ ]:
plt.figure(figsize=(5,3))
sns.barplot(data=df, x='clerkship', hue='clerkship', y='qual')
_ = plt.xticks(rotation=35)

## Model Performance

In [ ]:
# Make a copy of the dataframe for analysis
dfa = df.copy()
target_col = 'qual' # target
text_col = 'text' # text
print('DF original shape:', dfa.shape)
dfa = dfa.dropna(subset=[target_col, text_col])
print('DF new shape:', dfa.shape)

white_space_only = dfa[text_col].str.strip().str.len() == 0
white_space_replace = 'blank'
print(f'There are {white_space_only.sum()} ({white_space_only.sum()/len(white_space_only)*100:.2f}%) whitespace-only texts - replacing with "{white_space_replace}"')
dfa.loc[white_space_only, text_col] = white_space_replace

dfa = dfa.reset_index(drop=True)

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained('maxspad/nlp-qual-qual', model_max_length=512)
model = AutoModelForSequenceClassification.from_pretrained('maxspad/nlp-qual-qual')
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device='mps')

In [ ]:
res = pipe(dfa[text_col].tolist(), batch_size=16, truncation=True)

In [ ]:
dfr = (pd.concat([dfa, pd.json_normalize(res)], axis=1)
         .replace({ 'label': {
            'LABEL_5':5,
            'LABEL_4':4,
            'LABEL_3':3,
            'LABEL_2':2,
            'LABEL_1':1,
            'LABEL_0':0             
         }})
         .rename({
             'label': 'qual_pred_label',
             'score': 'qual_pred_score'
         }, axis=1))

In [ ]:
blah = dfr[['text','qual','qual_pred_label','qual_pred_score']]

In [ ]:
(blah['qual'] == blah['qual_pred_label']).sum() / len(blah)

# Subscores

Number of subscores available

In [ ]:
df[['evidence','suggestion','connection']].notna().sum()

Clerkships which rated their subscores

In [ ]:
df.groupby('clerkship')[['evidence','suggestion','connection']].agg(lambda x: x.notna().sum())

## Q1 - Evidence

Distribution of Evidence (Q1) subscore

In [ ]:
c = 'evidence'

In [ ]:
df[c].value_counts().sort_index()

In [ ]:
df[c].describe().reset_index().T

In [ ]:
plt.figure(figsize=(5,3))
sns.histplot(df[c])

In [ ]:
df.groupby('clerkship')[c].describe().dropna()

In [ ]:
plt.figure(figsize=(5,3))
sns.barplot(data=df.dropna(subset=c), y=c, x='clerkship', hue='clerkship')

## Suggestion (Q2)

Distribution of Suggestion (Q2) subscore

In [ ]:
c = 'suggestion'

In [ ]:
df[c].value_counts().sort_index()

In [ ]:
df[c].describe().reset_index().T

In [ ]:
plt.figure(figsize=(5,3))
sns.histplot(df[c])

In [ ]:
df.groupby('clerkship')[c].describe().dropna()

In [ ]:
plt.figure(figsize=(5,3))
sns.barplot(data=df.dropna(subset=c), y=c, x='clerkship', hue='clerkship')

## Connection (Q3)

Distribution of Connection (Q3) subscore

In [ ]:
c = 'connection'

In [ ]:
df[c].value_counts().sort_index()

In [ ]:
df[c].describe().reset_index().T

In [ ]:
plt.figure(figsize=(5,3))
sns.histplot(df[c])

In [ ]:
df.groupby('clerkship')[c].describe().dropna()

In [ ]:
plt.figure(figsize=(5,3))
sns.barplot(data=df.dropna(subset=c), y=c, x='clerkship', hue='clerkship')